In [1]:
!pip install tpot

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tpot import TPOTClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

C:\Users\Jose\anaconda3\Lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [3]:
dataset = pd.read_csv('credit_simple.csv', sep=';')
dataset.shape

(1000, 8)

In [4]:
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [5]:
y = dataset['CLASSE']
X = dataset.iloc[:,:-1]

In [6]:
#verificar se há dados faltantes
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [7]:
#preenchendo dados faltantes em SALDO_ATUAL pela sua mediana
mediana = X['SALDO_ATUAL'].median()
mediana

2323.0

In [8]:
X['SALDO_ATUAL'].fillna(mediana,inplace=True)
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [9]:
#para preencher os dados faltantes de ESTADOCIVIL (categórico) irei utilizar a sua MODA
agrupado = X.groupby(['ESTADOCIVIL']).size()
agrupado

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

In [10]:
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

In [11]:
#verifico há outliers no campo SALDO_ATUAL. Vou usar a técnica do DESVIO PADRAO e substituir por sua MEDIANA
desv = X['SALDO_ATUAL'].std()
desv

685936688.9820064

In [12]:
#encontrei indicios nos registros de indice 127 e 160. vou susbstituir pela MEDIANA claculada.
X.loc[X['SALDO_ATUAL']>= 2 * desv, 'SALDO_ATUAL']

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

In [13]:
mediana = X['SALDO_ATUAL'].median()
mediana

2323.0

In [14]:
X.loc[X['SALDO_ATUAL']>= 2 * desv, 'SALDO_ATUAL'] = mediana
X.loc[X['SALDO_ATUAL']>= 2 * desv]

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO


In [15]:
#Transformando dados poucos representados em categoria unica (BIDIN)
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

In [16]:
X.loc[X['PROPOSITO']=='Eletrodomésticos','PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO']=='qualificação','PROPOSITO'] = 'outros'
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  33
radio/tv               280
dtype: int64

In [17]:
#extração de características
X['DATA']

0      01/01/2019
1      01/01/2020
2      02/01/2020
3      02/01/2019
4      03/01/2018
          ...    
995    29/06/2018
996    30/06/2018
997    03/07/2018
998    04/07/2019
999    05/07/2018
Name: DATA, Length: 1000, dtype: object

In [18]:
X['DATA'] = pd.to_datetime(X['DATA'], format='%d/%m/%Y')

In [19]:
X['DATA']

0     2019-01-01
1     2020-01-01
2     2020-01-02
3     2019-01-02
4     2018-01-03
         ...    
995   2018-06-29
996   2018-06-30
997   2018-07-03
998   2019-07-04
999   2018-07-05
Name: DATA, Length: 1000, dtype: datetime64[ns]

In [20]:
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()

In [21]:
X['ESTADOCIVIL'].unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [22]:
X['PROPOSITO'].unique()

array(['radio/tv', 'educação', 'mobilia/equipamento', 'carro novo',
       'carro usado', 'negócios', 'outros', 'obras'], dtype=object)

In [23]:
X['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [24]:
labelencoder1 = LabelEncoder()
X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA']) 

In [25]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6


In [26]:
outros = X['OUTROSPLANOSPGTO'].unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

In [27]:
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix = 'OUTROS')

In [28]:
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
995,0,1,0
996,0,1,0
997,0,1,0
998,0,1,0


In [29]:
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6
...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,3,2018,6,0
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2
997,804.0,4,38,nenhum,2018-07-03,3,7,2018,7,5
998,1845.0,4,23,nenhum,2019-07-04,3,7,2019,7,4


In [30]:
sc = StandardScaler()
m = sc.fit_transform(X.iloc[:,0:3])
m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]])

In [31]:
X = pd.concat([X,z,pd.DataFrame(m,columns=['SALDO_ATUAL_N','RESIDENCEADESDE_N','IDADE_N'])],axis=1)

In [32]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCEADESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5,0,1,0,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6,0,1,0,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4,0,1,0,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6,0,1,0,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6,0,1,0,0.572709,1.046987,0.898982


In [33]:
X.drop(columns=['SALDO_ATUAL','RESIDENCIADESDE','IDADE','OUTROSPLANOSPGTO','DATA','OUTROS_banco'], inplace=True)

In [34]:
X

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCEADESDE_N,IDADE_N
0,3,7,2019,1,5,1,0,-0.745516,1.046987,1.639276
1,0,7,2020,1,6,1,0,0.957740,-0.765977,-0.740241
2,3,2,2020,1,4,1,0,-0.415337,0.140505,0.687469
3,3,3,2019,1,6,1,0,1.645526,1.046987,0.475956
4,3,0,2018,1,6,1,0,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...
995,0,3,2018,6,0,1,0,-0.543562,1.046987,-0.264338
996,2,1,2018,6,2,1,0,0.211898,1.046987,0.211566
997,3,7,2018,7,5,1,0,-0.875522,1.046987,0.105809
998,3,7,2019,7,4,1,0,-0.504738,1.046987,-0.687363


In [35]:
X_treinamento, X_teste, y_treinamento, y_teste =  train_test_split(X,y, test_size=0.3, random_state=0)

In [48]:
tpot = TPOTClassifier(
    generations = 100, #numero de geracoes que ele vai criar
    population_size = 100, # numero de configuracoes vai gerar para cada algoritomo gerado (valor padrão)
    offspring_size = 100, # numeros de dencendentes que vai ser gerado (valor padrão)
    mutation_rate =0.9, # taxa de mutação (valor padrão. É a probabilidade de algum elemento receber um valor aleatório)
    crossover_rate=0.1, # valor padrão. é a probabilidade de mistar as soluções
    scoring='accuracy', # metrica de avaliacao dos modelos
    max_time_mins=2, # tempo maximo em minutos por processo de avaliação. definido em 2 min.
    random_state=0, #
    early_stop=False, #interrompe de forma prematura o processo de avaliação
    verbosity=2
)

In [49]:
tpot.fit(X_treinamento, y_treinamento)
print("Melhor Modelo: ", tpot.fitted_pipeline_)

Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7028571428571428

The optimized pipeline was not improved after evaluating False more generations. Will end the optimization process.

TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: DecisionTreeClassifier(PCA(input_matrix, iterated_power=6, svd_solver=randomized), criterion=gini, max_depth=7, min_samples_leaf=2, min_samples_split=18)
Melhor Modelo:  Pipeline(steps=[('pca',
                 PCA(iterated_power=6, random_state=0,
                     svd_solver='randomized')),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(max_depth=7, min_samples_leaf=2,
                                        min_samples_split=18,
                                        random_state=0))])


In [38]:
tpot.evaluated_individuals_

{'BernoulliNB(input_matrix, BernoulliNB__alpha=0.001, BernoulliNB__fit_prior=False)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 1,
  'internal_cv_score': 0.61},
 'GaussianNB(PolynomialFeatures(input_matrix, PolynomialFeatures__degree=2, PolynomialFeatures__include_bias=False, PolynomialFeatures__interaction_only=False))': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 2,
  'internal_cv_score': 0.49428571428571433},
 'LogisticRegression(input_matrix, LogisticRegression__C=0.0001, LogisticRegression__dual=False, LogisticRegression__penalty=l2)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 1,
  'internal_cv_score': 0.6942857142857143},
 'ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=True, ExtraTreesClassifier__criterion=entropy, ExtraTreesClassifier__max_features=0.

In [50]:
previsao = tpot.predict(X_teste)
acuracia = accuracy_score(previsao, y_teste)
print(f"Acurácia: {acuracia}")

Acurácia: 0.6966666666666667
